# Pneumonia-Treatment-RAG

In [1]:
!pip install PyPDF2
!pip install transformers torch
!pip install sentence_transformers
!pip install sentencepiece
!pip install langchain_openai
!pip install openai 
!pip install qdrant-client


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# **SentenceTransformer Embedding **

In [2]:
from sentence_transformers import SentenceTransformer
def get_sentenceTF_embeddings(sentences):
  model = SentenceTransformer('all-MiniLM-L6-v2')
  embeddings =[]
  for chunk in sentences:
    embeddings.append(model.encode(chunk))
  print(len(embeddings))
  return embeddings

def Embed_stenteceTF(sentence):
  model = SentenceTransformer('all-MiniLM-L6-v2')
  return model.encode(sentence)

d:\FYP\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# **Sementic Chuncking**

In [3]:
import torch
from transformers import BertTokenizer, BertModel
from sklearn.cluster import KMeans


def get_bert_embeddings(sentences):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased')

    # Tokenize sentences
    inputs = tokenizer(sentences, return_tensors='pt', padding=True, truncation=True, max_length=128)

    # Get BERT embeddings
    with torch.no_grad():
        outputs = model(**inputs)

    # Use the [CLS] token representation as the sentence embedding
    embeddings = outputs.last_hidden_state[:, 0, :].numpy()

    return embeddings

def semantic_chunking_with_attention(text, n_clusters=5, max_chunk_size=512):
    # Split the input text into sentences based on '.'
    sentences = [sentence.strip() for sentence in text.split('.') if sentence.strip()]

    # Get BERT embeddings for sentences
    embeddings = get_bert_embeddings(sentences)

    # Perform KMeans clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=5).fit(embeddings)
    labels = kmeans.labels_

    # Group sentences by cluster
    clusters = {}
    for sentence, label in zip(sentences, labels):
        if label not in clusters:
            clusters[label] = []
        clusters[label].append(sentence)

    # Create chunks from clusters with max_chunk_size constraint
    chunks = []
    for cluster in clusters.values():
        current_chunk = ""
        for sentence in cluster:
            if len(current_chunk) + len(sentence) + 1 <= max_chunk_size:  # +1 for the period
                current_chunk += sentence + ". "
            else:
                chunks.append(current_chunk.strip())
                current_chunk = sentence + ". "
        if current_chunk:
            chunks.append(current_chunk.strip())

    return chunks

# **Recursive Chunking**

In [4]:
def division_Chunk(text, chunk_size=400):
    sentences = text.split(".")
    chunk1 = ""
    chunks = []

    for sentence in sentences:
        word_count = len(sentence.split())

        if word_count > chunk_size:
            chunks.extend(Recursive_Chunking(sentence, chunk_size))
        else:
            if len(chunk1.split()) + word_count <= chunk_size:
                chunk1 += sentence + "\n"
            else:
                chunks.append(chunk1.strip())
                chunk1 = sentence + "\n"

    if chunk1.strip():
        chunks.append(chunk1.strip())

    return chunks

def Recursive_Chunking(text, chunk_size=1000):
    chunks = text.split("\n\n")
    #print(len(chunks))
    result_chunks = []

    for chunk in chunks:
        word_count = len(chunk.split())

        if word_count > chunk_size:
            middle = int(0.8 * word_count)
            words = chunk.split()
            part1 = " ".join(words[:middle])
            part2 = " ".join(words[middle:])

            result_chunks.extend(division_Chunk(part1, chunk_size))
            result_chunks.extend(division_Chunk(part2, chunk_size))
        else:
            result_chunks.append(chunk.strip())

    return result_chunks

# Add_Data_From_PDF_file

In [5]:
from PyPDF2 import PdfReader
import re

def read_pdf_file(file_path):
    points = []
    # Open the PDF file
    with open(file_path, 'rb') as file:
        pdf_reader = PdfReader(file)
        # Iterate through each page in the PDF
        for page in pdf_reader.pages:
            text = page.extract_text()
            if text:
                points.append(text.strip())
    return points

def replace_double_spaces(text):
    # Replace all occurrences of two consecutive spaces with '\xa0'
    cleaned_text = re.sub(r'\xa0',' ' , text)
    cleaned_text1 = re.sub(r'\u200b','' , cleaned_text)
    return cleaned_text1

def read_pdf_file_(file_path, Chunking_type, Max_Chunk_size, NumberOf_cluster=20):
    Pneumonia = read_pdf_file(file_path)
    for i in range(len(Pneumonia)):
        Pneumonia[i] = replace_double_spaces(Pneumonia[i])
    Pneumonia_Str = "\n".join(Pneumonia)
    if Chunking_type == "Recursive":
        chunks = Recursive_Chunking(Pneumonia_Str, chunk_size=Max_Chunk_size)
    elif Chunking_type == "Semantic":
        chunks = semantic_chunking_with_attention(Pneumonia_Str, n_clusters=NumberOf_cluster, max_chunk_size=Max_Chunk_size)
    embedding = get_sentenceTF_embeddings(chunks)
    return embedding, chunks

# **PrepareData**

In [6]:
# Read Data from pdf file
embeddings, chunks = read_pdf_file_(file_path='D:\FYP\Save Embeddings to Qdrant\GuideBook.pdf',Chunking_type='Semantic',Max_Chunk_size = 600,NumberOf_cluster = 20)
print(len(chunks))
print(len(embeddings))

<>:2: SyntaxWarning: invalid escape sequence '\F'
<>:2: SyntaxWarning: invalid escape sequence '\F'
C:\Users\HP\AppData\Local\Temp\ipykernel_15184\3491981667.py:2: SyntaxWarning: invalid escape sequence '\F'
  embeddings, chunks = read_pdf_file_(file_path='D:\FYP\Save Embeddings to Qdrant\GuideBook.pdf',Chunking_type='Semantic',Max_Chunk_size = 600,NumberOf_cluster = 20)


73
73
73


In [7]:
from qdrant_client.models import PointStruct
import uuid

# Create PointStruct objects
points = [
    PointStruct(
        id=str(uuid.uuid4()),  # Use a unique ID for each document
        vector=embedding,  # Ensure this is a list of floats
        payload={"text": doc}  # Optional metadata
    )
    for i, (doc, embedding) in enumerate(zip(chunks, embeddings))
]

In [8]:
from qdrant_client import QdrantClient

url =  "https://e7ca7f9e-dadf-4ad1-9018-d709ee421a85.europe-west3-0.gcp.cloud.qdrant.io:6333"
      
api_key = "aXKv5m75swS2trKxdfGwhWSgkEPvaO4z1P-IYwIGj5kuMvgdA3zayQ"

# Initialize Qdrant client
qdrant_client = QdrantClient(url=url, api_key=api_key)


In [22]:
from qdrant_client.models import CollectionParams, VectorParams

# Define the dimension of your vectors and distance metric
vector_dim = 384  # Dimension of your vectors
distance_metric = "Cosine"  # Distance metric (use "Cosine" as string)

# Define vector configuration
vector_params = VectorParams(size=vector_dim, distance=distance_metric)

# Create the collection
qdrant_client.create_collection(
    collection_name="chatbot_vd",
    vectors_config=vector_params  # Correct argument name
)

UnexpectedResponse: Unexpected Response: 409 (Conflict)
Raw response content:
b'{"status":{"error":"Wrong input: Collection `chatbot_vd` already exists!"},"time":0.023230446}'

In [21]:
# Insert points into Qdrant
qdrant_client.upsert(
    collection_name="chatbot_vd",
    points=points
)

ResponseHandlingException: Server disconnected without sending a response.

In [11]:
query = ["how to treat leg fractures?"]
query_embedding_response = get_sentenceTF_embeddings(query)
# print(query_embedding_response)


1


In [12]:
# Assuming query_embedding_response is a list of lists or a NumPy array
query_embedding = query_embedding_response[0].tolist()  # Convert to list if necessary

In [13]:
# Perform similarity search
results = qdrant_client.search(
    collection_name="chatbot_vd",
    query_vector=query_embedding,

)
# Print results
for result in results:
    print(result)

UnexpectedResponse: Unexpected Response: 404 (Not Found)
Raw response content:
b'404 page not found\n'